In [1]:
from models import *
from config import *

langs = LanguageModel(config)
metar = MetaReasoner(config)

output_seqs = langs([
    "what is that ?",
    "what is Banach-Steinhaus theorem ?",
    "what is that concept ?"])



for s in output_seqs["seq_features"]:
    print(s.shape)




[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 54] Connection reset by peer>


AttributeError: 'Namespace' object has no attribute 'domain'